In [14]:

import main
from pars_shocks import Pars, Shocks
import plot_inequality as plot_ineq
import time
import importlib
import io_manager
import numpy as np
importlib.reload(plot_ineq)
#run stuff here
start_time = time.perf_counter()
print("Running main")

# ***** may want to change how trans is generated its redundant in do file.
of_name = None
main_path = "C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/model_uncert/"
age_type_input_path = main_path + "input/50p_age_moms/"

trans_full_path = age_type_input_path + "MH_trans_by_MH_clust_age.csv"  
trans_uncond_age_path = age_type_input_path + "MH_trans_uncond_age.csv"

H_type_pop_share_path = age_type_input_path + "MH_clust_50p_age_pop_shares.csv"

# do_phi_H_calib = True
do_phi_H_calib = False

myPars = main.pars_factory(main_path, H_type_pop_share_path = H_type_pop_share_path)
myPars.H_trans = io_manager.read_and_shape_H_trans_full(myPars, path = trans_full_path) 
# myPars.H_trans = io_manager.read_and_shape_H_trans_uncond_age(myPars, trans_uncond_age_path)
myPars, myShocks, sols, sims = main.main_io(main_path, myPars = myPars, out_folder_name = of_name, 
                                            do_phi_H_calib = do_phi_H_calib, output_flag = False)


Running main
Using default health transition matrix
*****Running main_io with default out_folder_name*****
Solver ran in 0.45062740000139456 seconds
Calibrating with alpha_lab_targ = 0.33496439, w0_mean_targ = 9.4541645, w0_sd_targ = 3.2013602, 
                                        w1_targ = 0.3406973883314617, w2_targ = 0.3031905120658842, wH_targ = 0.035280712, phi_H_target = 0.02408891
***** Calibration iteration 0 *****
***** Calibration iteration 1 *****
***** Calibration iteration 2 *****
***** Calibration iteration 3 *****
***** Calibration iteration 4 *****
***** Calibration iteration 5 *****
***** Calibration iteration 6 *****
***** Calibration iteration 7 *****
***** Calibration iteration 8 *****
***** Calibration iteration 9 *****
***** Calibration iteration 10 *****
***** Calibration iteration 11 *****
***** Calibration iteration 12 *****
Calibration converged after 13 iterations
********** phi_H calibration was skipped **********
w0_weights = [0.28046962 0.59335698 0.12

In [18]:
import model_uncert as model
importlib.reload(model)
import my_toolbox as tb
importlib.reload(tb)

import io_manager as io
import solver
import simulate

from typing import Tuple, Dict
from numba import njit

def calib_phi_H(myPars: Pars, main_path: str, tol:float, target:float, phi_H_min:float, phi_H_max:float)-> Tuple[float, float, Dict[str, np.ndarray], Dict[str, np.ndarray]]:
    '''
    calibrates the phi_H parameter of the model to match the target difference in mean log hours worked between the bad MH and good MH states
    takes the following arguments:
    myPars: the parameters of the model
    main_path: the path to the main directory
    tol: the tolerance for the calibration
    target: the target difference in mean log hours worked between the bad MH and good MH states
    phi_H_min: the minimum value of the phi_H parameter
    phi_H_max: the maximum value of the phi_H parameter
    returns a tuple with the calibrated phi_H, the difference in mean log hours worked between the bad MH and good MH states, the state solutions and the simulations
    '''
    io.print_params_to_csv(myPars, path = main_path, file_name = "pre_phi_H_calib_params.csv")
    phi_H_mom = -999.999
    state_sols = {}
    sim_lc = {}

    # define the lambda function to find the zero of
    get_phi_H_diff = lambda new_phi_H: phi_H_moment_giv_phi_H(myPars, new_phi_H)[0] - target
    print("Entering bisection search")
    print(f"Calibration tolerance: {tol}")
    calibrated_phi_H = tb.bisection_search(get_phi_H_diff, phi_H_min, phi_H_max, tol, myPars.max_iters, myPars.print_screen)
    myPars.phi_H = calibrated_phi_H

    # solve, simulate model for the calibrated phi_H
    phi_H_mom, state_sols, sim_lc = phi_H_moment_giv_phi_H(myPars, calibrated_phi_H)
    io.print_params_to_csv(myPars, path = main_path, file_name = "phi_H_calib_params.csv")
    if myPars.print_screen >= 1:
        print(f"Calibration exited: phi_H = {calibrated_phi_H}, difference in mean log hours worked = {phi_H_mom}, target difference in mean log hours worked = {target}")

    return calibrated_phi_H, phi_H_mom, state_sols, sim_lc

def phi_H_moment_giv_phi_H(myPars: Pars, new_phi_H:float)-> Tuple[float, Dict[str, np.ndarray], Dict[str, np.ndarray]]:
    '''
    solves the model for a given phi_H and returns the phi_H moment, the mean phi_H, the state solutions and the simulations
    '''
    myPars.phi_H = new_phi_H
    shocks = Shocks(myPars)
    state_sols = solver.solve_lc(myPars)
    sim_lc = simulate.sim_lc(myPars, shocks, state_sols)
    lab_sim_lc = sim_lc['lab']
    return phi_H_moment(myPars, lab_sim_lc, shocks), state_sols, sim_lc

# def phi_H_moment(myPars: Pars, sims: Dict[str, np.ndarray], shocks: Shocks)-> float:
# @njit
def phi_H_moment(myPars: Pars, lab_sims: np.ndarray, shocks: Shocks)-> float:
    '''
    returns the difference in mean log hours worked between the bad MH and good MH states 
    '''
    # lab_sims: np.ndarray = sims['lab'][:, :, :, :myPars.J]
    # print(f"lab_sims mean by age: {np.mean(lab_sims, axis = tuple(range(1, lab_sims.ndim - 1)))}")
    pre_retirement_age = 55
    pre_retirement_age_ind = np.where(myPars.age_grid == pre_retirement_age)[0][0]

    lab_sims = lab_sims[:, :, :, :myPars.J]
    H_hist: np.ndarray = shocks.H_hist[:, :, :, :myPars.J]
    # lab_sims = lab_sims[:, :, :, :pre_retirement_age_ind]
    # H_hist: np.ndarray = shocks.H_hist[:, :, :, :pre_retirement_age_ind]

    # take the log where hours are positive
    # lab_non_zero_mask = (lab_sims >= 0)
    for i in range(lab_sims.size):
        if lab_sims.flat[i] <= 0:
            lab_sims.flat[i] = 1e-10

    # log_lab_sims = np.log(lab_sims, where = lab_sims > 0)
    # log_lab_sims = np.log(lab_sims * lab_non_zero_mask)

    log_lab_sims = np.log(lab_sims)
    # log_lab_sims = lab_sims
    

    bad_MH_log_lab_sims = log_lab_sims * (H_hist == 0)
    good_MH_log_lab_sims = log_lab_sims * (H_hist == 1)

    bad_MH_log_lab_mean = model.wmean_non_zero(myPars, bad_MH_log_lab_sims)
    good_MH_log_lab_mean = model.wmean_non_zero(myPars, good_MH_log_lab_sims)
    # bad_MH_log_lab_mean = np.mean(bad_MH_log_lab_sims, where = bad_MH_log_lab_sims != 0)
    # good_MH_log_lab_mean = np.mean(good_MH_log_lab_sims, where = good_MH_log_lab_sims != 0)
    diff = good_MH_log_lab_mean - bad_MH_log_lab_mean   

    # print(f"good_MH_log_lab_mean: {good_MH_log_lab_mean} - bad_MH_log_lab_mean: {bad_MH_log_lab_mean} = diff: {diff}")

    return diff
    
def get_phi_H_targ(myPars: Pars, target_folder_path: str)-> float:
    '''
    reads phi_H target moment from myPars.path + '/input/MH_hours_moments.csv'
    '''
    # data_moments_path = myPars.path + '/input/MH_labor_moments.csv'
    data_moments_path = target_folder_path + '/MH_hours_moments.csv'
    data_mom_col_ind = 0
    mean_log_lab_diff = tb.read_specific_column_from_csv(data_moments_path, data_mom_col_ind)
    return mean_log_lab_diff[0]

In [19]:
print_screen = 2
myPars.print_screen = print_screen
phi_H_min = 0.00001
phi_H_max = 0.5
targ_input_path = age_type_input_path 
# phi_H_mom_targ = get_phi_H_targ(myPars, targ_input_path)
phi_H_mom_targ = 0.03
calib_tol = 1e-2 # standard value is 1e-3

# new_wH_myPars = myPars
# new_wH = np.log(1.0)
# new_wH_myPars.wH_coeff = new_wH

# new_phi_H = 0.1
phi_arr = np.arange(0.001, 0.030, 0.005)
print(f"phi_arr: {phi_arr}")

# for new_phi_H in phi_arr:
#     phi_H_mom = phi_H_moment_giv_phi_H(myPars, new_phi_H)[0]
    # print(f"phi_H_mom: {phi_H_mom}")

# phi_H_mom = phi_H_moment_giv_phi_H(myPars, new_phi_H)[0]
# print(f"phi_H_mom: {phi_H_mom}")

print(f"phi_H_mom_targ: {phi_H_mom_targ}")
calib_phi_H, calib_phi_H_moment, calib_state_sols, calib_sim_lc = calib_phi_H(myPars, main_path, calib_tol, phi_H_mom_targ, phi_H_min, phi_H_max)

phi_arr: [0.001 0.006 0.011 0.016 0.021 0.026]
phi_H_mom_targ: 0.03
Entering bisection search
Calibration tolerance: 0.01
Bisection search: returning low end point
Calibration exited: phi_H = 1e-05, difference in mean log hours worked = 0.03198648377521374, target difference in mean log hours worked = 0.03
